# 데이터분석 에이전트: 눈깜짝할 사이에 데이터에서 인사이트 얻기 ✨
_저자: [Aymeric Roucher](https://huggingface.co/m-ric)_
번역: [안정](https://huggingface.co/Ahnj-Stability)_

> 이 튜토리얼은 고급 과정입니다. 사전에 [이 쿡북](agents)에 대한 개념을 이해하고 있어야 합니다!

이번에 만들 **데이터 분석 에이전트**는 :
 **데이터 분석 라이브러리를 사용한 코드 에이전트로,데이터프레임을 불러오고 변환해 데이터로부터 인사이트를 찾아내고,결과를 시각화합니다!**


예를 들어 여러분이 '[Kaggle Titanic 챌린지](https://www.kaggle.com/competitions/titanic)'의 데이터를 직접 분석해 개별 승객의 생존 여부를 예측하고 싶다고 가정해 봅시다.
하지만 직접 분석에 들어가기 전에, 자율 에이전트가 수치도 추출해주고 그에 맞게 그래프도 그려주어 인사이트를 뽑아내주면 좋겠군요.

이 시스템을 설정해 보겠습니다.

아래 명령어를 실행하여 필요한 의존성을 설치하세요:

In [ ]:
!pip install seaborn "transformers[agents]"

우선 에이전트를 생성합니다. 이 쿡북에서는 `ReactCodeAgent`를 사용했습니다. (더 많은 종류의 에이전트를 확인하려면 [이 문서](https://huggingface.co/docs/transformers/en/agents) 를 참조하세요.)
이 에이전트는 별도의 도구를 제공하지 않아도 스스로 코드를 작성하고 실행할 수 있습니다.

일반적으로 `additional_authorized_imports`에 라이브러리를 전달할 때, 파이썬 인터프리터는 환경에 설치된 라이브러리만 사용할 수 있기 때문에, 해당 라이브러리들이 로컬 환경에 설치되어 있는지 확인해야 합니다.


⚙ 해당 에이전트는 [meta-llama/Meta-Llama-3.1-70B-Instruct](https://huggingface.co/meta-llama/Meta-Llama-3.1-70B-Instruct) 모델을 사용하며, HF의 Inference API를 사용하는 `HfEngine` 클래스로 구동됩니다. Inference API를 통해 빠르고 쉽게 OS 모델을 실행할 수 있습니다.

In [ ]:
from transformers.agents import HfEngine, ReactCodeAgent
from huggingface_hub import login
import os

login(os.getenv("HUGGINGFACEHUB_API_TOKEN"))
llm_engine = HfEngine("meta-llama/Meta-Llama-3.1-70B-Instruct")

agent = ReactCodeAgent(
    tools=[],
    llm_engine=llm_engine,
    additional_authorized_imports=["numpy", "pandas", "matplotlib.pyplot", "seaborn"],
    max_iterations=10,
)

## 데이터 분석 📊🤔
저자는 구글 코랩을 사용하였으므로, 드라이브를 마운트하고 분석에 사용할 디렉토리를 생성했습니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')  # 구글 드라이브 마운트

In [ ]:
import os

os.chdir('/content/drive/My Drive/Colab Notebooks')  # 원하는 경로로 변경
print(os.getcwd())  # 변경된 경로 확인

In [ ]:
# 디렉토리 생성
os.mkdir("./figures")

In [ ]:
os.chdir('/content/drive/My Drive/Colab Notebooks/figures')  # 생성한 디렉토리 경로로 변경
print(os.getcwd())  # 변경된 경로 확인

실제 캐글 대회에서 사용된 추가 노트를 에이전트를 실행할 때 `run`메소드의 kwarg로 넘겨주었습니다.

In [ ]:
additional_notes = """
### Variable Notes
승객ID : 승객 아이디
생존여부 : (0: 사망, 1: 생존)
클래스: 사회경제적 지위
1 = 상류층
2 = 중산층
3 = 하류층
성별 : 남성, 여성
나이 : 나이가 1 미만일 경우 소수로 표시됩니다. 나이가 추정된 경우, xx.5 형태로 표시됩니다.
형제배우자: 이 데이터셋은 가족 관계를 다음과 같이 정의합니다...
형제자매 = 형제, 자매, 이복형제, 이복자매
배우자 = 남편, 아내 (애인과 약혼자는 간주하지 않았습니다.)
부모자식: 이 데이터셋은 가족 관계를 다음과 같이 정의합니다...
부모 = 어머니, 아버지
자식 = 딸, 아들, 양녀, 양자
유모와 여행을 온 몇 어린이들의 경우, 부모자식=0 로 표현합니다.
요금: 티켓 요금
"""

analysis = agent.run(
    """당신은 데이터 분석 전문가입니다. 소스 파일을 로드하고 내용을 분석해주세요.
먼저 주어진 변수와 생존율과의 상관관계에 대해 흥미로운 질문 3가지를 나열하고, 해당 질문에 하나씩 답하고 관련된 숫자를 찾아내세요.
matplotlib/seaborn을 사용해 3가지 질문에 관련된 그래프를 그리고, './figures/' 폴더에 저장하십시오. 다른 그래프를 그리기 전에 plt.clf()로 각 그래프를 지워야 합니다. 그래프의 라벨명은 영어로 해주세요.

최종 답변 :  숫자로부터 찾은 상관관계와 인사이트를 요약해주세요.
(인사이트 예시: "is_december와 boredness 간의 상관관계는 1.3453으로, 이는 사람들이 겨울에 더 지루함을 느낄 가능성이 높다는 것을 시사합니다.")
최종 답변은 최소한 3개의 번호가 매겨진 항목이어야합니다.
""",
    additional_notes=additional_notes,
    source_file="titanic_ko/train.csv",
)

In [21]:
print(analysis)

성별과 생존여부 간의 상관관계: 
남성: 0.188908
여성: 0.742038
클래스와 생존여부 간의 상관관계: 
1등급: 0.629630
2등급: 0.472826
3등급: 0.242363
나이와 생존여부 간의 상관관계: -0.077221


놀랍지 않나요? 에이전트에게 시각화 도구를 제공해 자신이 만든 그래프를 분석하게 할 수도 있습니다!

## 데이터과학자 에이전트 : 예측 수행하기 🛠️

👉 이제 더 깊이 들어가 봅시다 : **데이터를 기반으로 모델이 예측을 수행하도록 합니다.**

예측 수행을 위해 `additional_authorized_imports`에 `sklearn`도 추가해줍니다.

In [ ]:
agent = ReactCodeAgent(
    tools=[],
    llm_engine=llm_engine,
    additional_authorized_imports=[
        "numpy",
        "pandas",
        "matplotlib.pyplot",
        "seaborn",
        "sklearn",
    ],
    max_iterations=12,
)

output = agent.run(
    """당신은 전문가 수준의 머신러닝 엔지니어입니다.
'titanic_ko/train.csv' 파일을 사용하여 '생존여부'를 예측하는 머신러닝 모델을 학습시키세요.
'titanic_ko/test.csv' 파일의 행에 대한 예측을 수행한 후, 결과를 'titanic_ko/output.csv'에 출력하세요.
함수와 모듈을 사용하기 전에 반드시 임포트하세요!
""",
    additional_notes=additional_notes + "\n" + analysis,
)

위에서 에이전트가 출력한 테스트 예측을 Kaggle에 제출하면 **0.78229**으로, 이는 17,360명 중 2824등에 해당하며, 저자가 몇 년 전 이 데이터분석 챌린지를 처음 시도해 힘들게 얻었던 결과보다 더 나은 성과입니다.

결과는 다를 수 있지만, 몇 초 만에 에이전트를 사용해 이 정도 성과를 낼 수 있다는 점이 매우 인상적입니다.

🚀 이것은 에이전트를 활용한 단순한 데이터 분석 사례일 뿐입니다. 사용 사례에 맞게 충분히 개선할 수 있습니다 🤗